In [3]:
#目前要做的工作
""" 
将重排序模型BCE转化为tensorrt engine

将向量模型转化为tensorrt engine

压缩模型可能需要自己实现论文算法与tensorrt引擎进行匹配

"""

' \n将重排序模型BCE转化为tensorrt engine\n\n将向量模型转化为tensorrt engine\n\n压缩模型可能需要自己实现论文算法与tensorrt引擎进行匹配\n\n'

In [3]:
import os
file_dir=os.getcwd()
import configparser
config = configparser.ConfigParser()
file_dir=os.path.join(file_dir,'../')
config.read('../cache.ini')
file_name=config['settings']['HF_HOME']
file_path=os.path.join(file_dir,file_name)
print(file_path)

/root/autodl-tmp/3.0/Tensorrt/resources/../../Engine/model/.cache


In [2]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION']='python'
import configparser
config = configparser.ConfigParser()
file_dir=os.getcwd()
file_dir=os.path.join(file_dir,'../')
config.read('../cache.ini')
os.environ["HF_DATASETS_CACHE"] = os.path.join(file_dir,config['settings']['HF_DATASETS_CACHE'])
os.environ["HF_HOME"] =os.path.join(file_dir,config['settings']['HF_HOME'])
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(file_dir,config['settings']['HUGGINGFACE_HUB_CACHE'])
os.environ["TRANSFORMERS_CACHE"] = os.path.join(file_dir,config['settings']['TRANSFORMERS_CACHE'])
os.environ["HF_ENDPOINT"] =config['settings']['HF_ENDPOINT']
os.environ["XDG_CACHE_HOME"] = os.path.join(file_dir,config['settings']['XDG_CACHE_HOME'])
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
# list of sentences
import torch
torch.backends.cuda.enable_flash_sdp(enabled=True)
torch.backends.cuda.enable_mem_efficient_sdp(enabled=True)
torch.backends.cuda.enable_math_sdp(enabled=True)
tokenizer = AutoTokenizer.from_pretrained('maidalun1020/bce-embedding-base_v1',cache_dir=os.path.join(file_dir,config['settings']['TRANSFORMERS_CACHE']))
model = AutoModel.from_pretrained('maidalun1020/bce-embedding-base_v1',torch_dtype=torch.float32,device_map='auto',cache_dir=os.path.join(file_dir, config['settings']['TRANSFORMERS_CACHE']))
model.eval()
#device = 'cuda'  # if no GPU, set "cpu"
#model.to(device)

/root/miniconda3/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/rag/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-12-13 14:02:48.405943: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-13 14:02:48.410487: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or d

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine

In [2]:
model.eval()

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine

In [3]:
!pip install onnx onnxruntime pycuda

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.
tensorflow-gpu 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.


In [3]:
import tensorrt as trt

In [5]:
device="cuda"
def embedding(text_list):
    inputs = tokenizer(text_list, padding=True, truncation=True, max_length=520, return_tensors="pt")
    inputs_on_device = {k: v.to(device) for k, v in inputs.items()}
    # get embeddings
    with torch.no_grad():
        outputs = model(**inputs_on_device, return_dict=True)
    return outputs

In [6]:
outputs=embedding(["你好，好久不见"])
import numpy as np


In [7]:
outputs[1].shape

torch.Size([1, 768])

In [8]:
outputs[0].shape

torch.Size([1, 7, 768])

In [9]:
inputs = tokenizer(["Hello, my dog is cute"], padding=True, truncation=True, max_length=520, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[    0, 35378,     4,   759, 10269,    83, 99942,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [4]:
# 将 PyTorch 模型转化为 ONNX 引擎
# 1、定义输入张量的形状信息
import numpy as np
import torch

# 创建输入张量
input_id_ = torch.randint(2, 1000, (1, 512),dtype=torch.int64).to('cuda')  # 在 GPU 上创建 input_ids
attention_mask_ = torch.ones((1, 512), dtype=torch.int64).to('cuda')  # 正确创建 attention_mask 并转到 GPU

# 转化模型
torch.onnx.export(
    model,  # 原模型
    (input_id_, attention_mask_),  # 输入张量，接受一个张量或者元组
    "engine/BCEembedding.onnx",
    export_params=True,  # 是否保存模型的权重信息
    opset_version=17,  # 17支持 INormalizationLayer，防止溢出
    do_constant_folding=True,  # 是否执行常量折叠优化
    input_names=['input_ids', 'attention_mask'],  # 输入的名字
    output_names=['last_hidden_state', 'pooler_output'],  # 输出的名字
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_length'},
        'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
        'last_hidden_state': {0: 'batch_size', 1: 'sequence_length'},
        'pooler_output': {0: 'batch_size'},
    }  # 可变长度，在 NLP 中批次和序列长度都是可变长度
)


In [5]:
import tensorrt as trt
logger=trt.Logger(trt.Logger.WARNING)
trt.init_libnvinfer_plugins(logger,namespace='')

True

In [6]:
builder=trt.Builder(logger)

In [7]:
config=builder.create_builder_config()

In [8]:
config.set_flag(trt.BuilderFlag.FP16)

In [17]:
network = builder.create_network()

[12/10/2024-11:03:03] [TRT] [W] The implicit batch dimension mode has been deprecated. Please create the network with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag whenever possible.


In [9]:
network=builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

In [10]:
profile = builder.create_optimization_profile()
profile.set_shape("input_ids", (1, 1),(4, 256),(8, 512))  # 输入的最小、默认批量大小、最大批次
profile.set_shape("attention_mask",(1,1),(4,256),(8,512))
config.add_optimization_profile(profile)

0

In [11]:
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE,1<<30)

In [12]:
parser=trt.OnnxParser(network,logger)

In [13]:
success=parser.parse_from_file('engine/BCEembedding.onnx')

[libprotobuf WARNING google/protobuf/io/coded_stream.cc:604] Reading dangerously large protocol message.  If the message turns out to be larger than 2147483647 bytes, parsing will be halted for security reasons.  To increase the limit (or to disable these warnings), see CodedInputStream::SetTotalBytesLimit() in google/protobuf/io/coded_stream.h.
[libprotobuf WARNING google/protobuf/io/coded_stream.cc:81] The total number of bytes read was 1112412160


[12/13/2024-14:04:14] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[12/13/2024-14:04:14] [TRT] [W] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped


[libprotobuf WARNING google/protobuf/io/coded_stream.cc:604] Reading dangerously large protocol message.  If the message turns out to be larger than 2147483647 bytes, parsing will be halted for security reasons.  To increase the limit (or to disable these warnings), see CodedInputStream::SetTotalBytesLimit() in google/protobuf/io/coded_stream.h.
[libprotobuf WARNING google/protobuf/io/coded_stream.cc:81] The total number of bytes read was 1112412160


In [14]:
print(success)

True


In [11]:
for idx in range(parser.num_errors):
    print(parser.get_error(idx))
if not success:
    pass

/root/autodl-tmp/3.0/Tensorrt/resources/../../Engine/model/.cache


In [15]:
serialized_engine=builder.build_serialized_network(network,config)

[12/13/2024-14:05:04] [TRT] [W] TensorRT encountered issues when converting weights between types and that could affect accuracy.
[12/13/2024-14:05:04] [TRT] [W] If this is not the desired behavior, please modify the weights or retrain with regularization to adjust the magnitude of the weights.
[12/13/2024-14:05:04] [TRT] [W] Check verbose logs for the list of affected weights.
[12/13/2024-14:05:04] [TRT] [W] - 118 weights are affected by this issue: Detected subnormal FP16 values.
[12/13/2024-14:05:04] [TRT] [W] - 46 weights are affected by this issue: Detected values less than smallest positive FP16 subnormal value and converted them to the FP16 minimum subnormalized value.
[12/13/2024-14:05:04] [TRT] [W] - 1 weights are affected by this issue: Detected finite FP32 values which would overflow in FP16 and converted them to the closest finite FP16 value.


In [16]:
with open('engine/BCEembedding_engine','wb') as f:
          f.write(serialized_engine)

In [17]:
import os
#读取tensorrt引擎
import tensorrt as trt
logger=trt.Logger(trt.Logger.INFO)
runtime=trt.Runtime(logger)
trt.init_libnvinfer_plugins(logger,'')
with open('engine/BCEembedding_engine','rb') as f:
    serialized_engine=f.read()
    engine=runtime.deserialize_cuda_engine(serialized_engine)

[12/13/2024-14:05:52] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[12/13/2024-14:05:52] [TRT] [I] Loaded engine size: 532 MiB
[12/13/2024-14:05:52] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +530, now: CPU 0, GPU 530 (MiB)


In [18]:
for i in range(engine.num_io_tensors):
    tensor_name=engine.get_tensor_name(i)
    tensor_shape=engine.get_tensor_shape(tensor_name)
    size=trt.volume(tensor_shape)
    dtype=trt.nptype(engine.get_tensor_dtype(tensor_name))
    
    print(f"Tensor Name:{tensor_name},Shape:{tensor_shape},Size:{size},Data Type:{dtype}")

Tensor Name:input_ids,Shape:(-1, -1),Size:1,Data Type:<class 'numpy.int32'>
Tensor Name:attention_mask,Shape:(-1, -1),Size:1,Data Type:<class 'numpy.int32'>
Tensor Name:last_hidden_state,Shape:(-1, -1, 768),Size:768,Data Type:<class 'numpy.float32'>
Tensor Name:pooler_output,Shape:(-1, 768),Size:-768,Data Type:<class 'numpy.float32'>


In [19]:
import numpy as np
input_data=np.arange(1,4,dtype=np.int64)
attention_mask=np.ones((1,4),dtype=np.int64)
last_hidden_state=np.empty((1,4,768),dtype=np.float32)
pooler_output=np.empty((1,768),dtype=np.float32)

In [20]:
import tensorrt

In [45]:
inputs = tokenizer("你好", padding=True, truncation=True, max_length=520,return_tensors="pt")
inputs['input_ids'].device

device(type='cpu')

In [21]:

import os
#CUDA_VISIBLE_DEVICES=0
#os.environ['CUDA_VISIBLE_DEVICES']='0'
#读取tensorrt引擎
import pycuda.driver as cuda
import pycuda.autoinit
import tensorrt as trt
logger=trt.Logger(trt.Logger.INFO)
runtime=trt.Runtime(logger)
trt.init_libnvinfer_plugins(logger,'')
with open('engine/BCEembedding_engine','rb') as f:
    serialized_engine=f.read()
    engine=runtime.deserialize_cuda_engine(serialized_engine)
import numpy as np
with engine.create_execution_context() as context:
    context.set_input_shape('attention_mask', (1,512))
    context.set_input_shape('input_ids', (1,512))
    input_data=np.arange(1,512,dtype=np.int64)
    attention_mask=np.ones((1,512),dtype=np.int64)
    last_hidden_state=np.empty((1,512,768),dtype=np.float32)
    pooler_output=np.empty((1,768),dtype=np.float32)
    d_input_ids=cuda.mem_alloc(input_data.nbytes)
    d_input_mask=cuda.mem_alloc(input_data.nbytes)
    d_last_hidden_state=cuda.mem_alloc(last_hidden_state.nbytes)
    d_pooler_output=cuda.mem_alloc(pooler_output.nbytes)
    context.set_tensor_address('input_ids', int(d_input_ids))
    context.set_tensor_address('attention_mask',int(d_input_mask))
    context.set_tensor_address('last_hidden_state', int(d_last_hidden_state))
    context.set_tensor_address('pooler_output', int(d_pooler_output))
    stream=cuda.Stream()
    cuda.memcpy_htod_async(d_input_ids,input_data,stream)
    cuda.memcpy_htod_async(d_input_mask,input_data,stream)
    bindings = [int(d_input_ids),int(d_input_mask),int(d_last_hidden_state),int(d_pooler_output)]
    context.execute_v2(bindings)
    cuda.memcpy_dtoh(last_hidden_state,d_last_hidden_state)

print(last_hidden_state[:,0])

[12/13/2024-14:05:59] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[12/13/2024-14:06:00] [TRT] [I] Loaded engine size: 532 MiB
[12/13/2024-14:06:00] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +530, now: CPU 0, GPU 1060 (MiB)
[12/13/2024-14:06:00] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +148, now: CPU 0, GPU 678 (MiB)
[[-2.35595703e-02 -2.06909180e-02  3.41796875e-02 -6.09588623e-03
   1.20727539e-01  5.86242676e-02 -1.54495239e-03  5.39245605e-02
   9.83886719e-02  4.61730957e-02 -6.53457642e-03  9.34448242e-02
   5.21545410e-02  1.25579834e-02  7.26318359e-03  4.48913574e-02
  -3.54003906e-02  4.78744507e-03 -7.43408203e-02  1.62719727e-01
  -6.06994629e-02  1.60827637e-02 -4.13894653e-03 -1.8676757

In [21]:
context=engine.create_execution_context()

[11/28/2024-08:54:53] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +52, now: CPU 0, GPU 634 (MiB)


In [22]:
import numpy as np
input_data=np.arange(1,512,dtype=np.int64)
attention_mask=np.ones((1,512),dtype=np.int64)
last_hidden_state=np.empty((1,512,768),dtype=np.float32)
pooler_output=np.empty((1,768),dtype=np.float32)

In [5]:
input_data.dtype

dtype('int64')

In [11]:
dtype=np.float32
output_data=np.empty((1,768),dtype=dtype)

In [5]:
stream=cuda.Stream()

In [6]:
cuda.memcpy_htod_async(d_input_ids,input_data,stream)
cuda.memcpy_htod_async(d_input_mask,input_data,stream)

In [7]:
context.set_tensor_address('input_ids', int(d_input_ids))
context.set_tensor_address('attention_mask',int(d_input_mask))
context.set_tensor_address('last_hidden_state', int(d_last_hidden_state))
context.set_tensor_address('pooler_output', int(d_pooler_output))

True

In [8]:
bindings = [int(d_input_ids),int(d_input_mask),int(d_last_hidden_state),int(d_pooler_output)]


In [29]:
type(bindings[0])

int

In [30]:
input_ids.shape

NameError: name 'input_ids' is not defined

In [9]:
context.set_input_shape("input_ids",(1,4))

True

True

In [11]:
print(bindings)

[123813383634944, 123813383635456, 123813383635968, 123813383648256]


In [12]:
context.execute_v2(bindings)

[11/28

False

/2024-08:57:00] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Myelin ([exec_instruction.cpp:exec:914] CUDA error 400 launching __myl_EqlNotCasTra kernel.)


In [ ]:
cuda.memcpy_dtoh(last_hidden_state,d_last_hidden_state)
print(last_hidden_state[:,0])

NameError: name 'cuda' is not defined